## Installation

In [ ]:
# Make sure your prompt starts with (venv)
pip install "pyats[full]" 

In [ ]:
# To upgrade
pip install pyats[full] --upgrade

In [ ]:
# To check the current version
pyats version check

## Basic Connectivity

In [ ]:
# Basic device connectivity

from genie.testbed import load

# Make sure the yaml file is in the same
# directory, or provide absolute path
tb = load('device.yaml')
dev = tb.devices['R1']
dev.connect()

# To disable default configuration
dev.connect(mit=True)

## Parsing

In [ ]:
# Using Bash
pyats parse "show version" --testbed-file device.yaml --devices router1

In [ ]:
# Multiple commands
pyats parse "show version" "show ip interface bried" --testbed-file device.yaml --devices router1

In [ ]:
# Using Python
from genie.testbed import load

tb = load('device.yaml')
dev = tb.devices['R1']
dev.connect()
p1 = dev.parse('show version')

print(p1)

In [ ]:
# Using Python
from genie.testbed import load

tb = load('device.yaml')
dev = tb.devices['R1']
dev.connect(mit=True) # change this to False to see the difference
p1 = dev.parse('show ip route')

dev = tb.devices['S1']
dev.connect(mit=True)
p2 = dev.parse('show vlan brief')

In [ ]:
# Using Python
from genie.testbed import load

routers = ('R1', 'R2', 'R3')

for router in routers:
    tb = load('device.yaml')
    dev = tb.devices[router]
    dev.connect(mit=True)
    p1 = dev.parse('show ip route')


## Learning Features

In [ ]:
# Using Python
from genie.testbed import load

tb = load('device.yaml')
dev = tb.devices['R1']
dev.connect()
p1 = dev.learn('ospf')

print(p1)

In [ ]:
# Using pyats interactive shell
(pyats) $ pyats shell --testbed-file device.yaml

# Python code in the next cell

In [ ]:
dev = testbed.devices['router1']
dev.connect()
output = dev.learn('ospf')

In [ ]:
# Using Bash
# You can redirect the output into a text file
# Replace ouput_folder
(pyats) $ pyats learn ospf --testbed-file device.yaml --devices router1 --output output_folder

In [ ]:
# To learn multiple features
(pyats) $ pyats learn ospf bgp --testbed-file device.yaml --devices router1 --output output_folder

In [ ]:
# To learn all features
(pyats) $ pyats learn all --testbed-file device.yaml --devices router1 --output output_folder

## Configuring Devices

### The direct method

In [ ]:
from genie.testbed import load

config_commands = '''
    interface Ethernet0/1
    no shutdown
'''

tb = load('device.yaml')
dev = tb.devices['R1']
dev.connect()
dev.configure(config_commands)

print(f"Configuration applied to R1")

dev.disconnect()

In [ ]:
# With exception handling
from genie.testbed import load

try:
    config_commands = '''
    interface Ethernet0/1
    no shutdown
    '''
    
    tb = load('device.yaml')
    dev = tb.devices['R1']
    dev.connect()
    dev.configure(config_commands)

    print(f"Configuration applied to R1")
    dev.disconnect()
    
except Exception as e:
    print("Error: ", str(e))

### Using Genie objects

In [ ]:
from genie.testbed import load
from genie.conf.base import Interface

tb = load('device.yaml')
uut = tb.devices['router1']

uut.connect()
interface = Interface(device=uut, name="Ethernet0/1")

interface.ipv4 = '192.168.0.1'
interface.ipv4.netmask = '255.255.255.0'
interface.shutdown = False

# Change to True if you want to apply configuration
print(interface.build_config(apply=False))

# To remove configuration
print(interface.build_unconfig(apply=True))


## Comparing Network States

In [ ]:
# Using Bash
(pyats) $ pyats learn "ospf" --testbed-file device.yaml --output output1

In [ ]:
# Repeat the 1st step, but save in a different file
(pyats) $ pyats learn "ospf" --testbed-file device.yaml --output output2

In [ ]:
# To compare possible changes
# Using Bash
$ (pyats) genie diff output1 output2

+ indicates an addition
- indicates a deletion
+ followed by - indicates a change

## Network Testing

In [ ]:
# shut and no shut bgp to see if the network behaves as expected
# In the cli, use the mock yaml file
(pyats) $ genie run --testbed-file mock.yaml --trigger-uids="TriggerShutNoShutBgp" --verification-uids="Verify_BgpProcessVrfAll" --devices uut

In [ ]:
# To view logs from the web panel
# In the cli
pyats logs view

## APIs

In [ ]:
from genie.testbed import load

testbed = load('device.yaml')
device = testbed.devices['R1']
device.connect()

# Display routing table
routes = device.api.get_routes()

# In a dict format
print(routes)

In [ ]:
# To shutdown an interface
device.api.shut_interface(interface='GigabitEthernet3')

In [ ]:
# To learn supported APIs on the connected device
dir(device.api)